In [7]:
# DO NOT delete this cell. 
# 
# This is the name of the file to be compressed.  
# Yes, you can create your own test cases and you should.

filename = "bible2.txt"

In [8]:
# LZW/ Your code 

# to deal with non ascii characters
def isASCII(text):
    # if text is a word, then it has to be made up of ASCII characters due to mechanics of LZW
    if (len(text) > 1):
        return 
    
    # if text is a single character, then check if its unicode is more than 256, and raise exception
    if (ord(text) > 256):
        raise Exception(text + " is not in ASCII")

def write(filename, listOfNumbers):
    #convert listOfNumbers from decimal to binary and store in binaryList
    binaryList = []
    for number in listOfNumbers:
        binaryList.append(bin(number)[2:].zfill(12))

    # concatinate all the binary strings in binaryList into one string object 
    # and divide them into to bytes and put in bytesList
    stringBuffer = ""
    bytesList = []
    for binaryNum in binaryList:
        stringBuffer += binaryNum

        # run this check twice because sometimes there's 1 byte, other times there's 2 bytes
        if(len(stringBuffer) >= 8):
            bytesList.append(stringBuffer[:8])
            stringBuffer = stringBuffer[8:]
        if(len(stringBuffer) >= 8):
            bytesList.append(stringBuffer[:8])
            stringBuffer = stringBuffer[8:]
    
    # if there are any bits left in stringBuffer, add 0s to make it 8 bits
    if (len(stringBuffer) > 0):
        stringBuffer += "0000"
        bytesList.append(stringBuffer)    

    # Convert binary strings to bytes
    binaryBytes = bytearray(int(b, 2) for b in bytesList)

    # Write bytes to file
    with open(filename, 'wb') as f:
        f.write(binaryBytes)

# write to file in binary, given a list of binary strings
def writeModified(filename, binaryList):
    # concatinate all the binary strings in binaryList into one string object 
    # and divide them into to bytes and put in bytesList
    stringBuffer = ""
    bytesList = []
    for binaryNum in binaryList:
        stringBuffer += binaryNum

        # run this check twice because sometimes there's 1 byte, other times there's 2 bytes
        if(len(stringBuffer) >= 8):
            bytesList.append(stringBuffer[:8])
            stringBuffer = stringBuffer[8:]
        if(len(stringBuffer) >= 8):
            bytesList.append(stringBuffer[:8])
            stringBuffer = stringBuffer[8:]
    
    # append 0s to make it a multiple of 8
    remainder = len(stringBuffer) % 8
    if remainder != 0:
        zeros_to_append = 8 - remainder
        stringBuffer += '0' * zeros_to_append

    # divide stringBuffer into 8-bit bytes and append to bytesList
    for i in range(0, len(stringBuffer), 8):
        bytesList.append(stringBuffer[i:i+8])
    
    # Convert binary strings to bytes
    binaryBytes = bytearray(int(b, 2) for b in bytesList)

    # Write bytes to file
    with open(filename, 'wb') as f:
        f.write(binaryBytes)

# read file in binary and convert to 12-bit ascii     
def read(filename = "txt.txt"):
    binaryContent = ""
    tempChar = ""
    numbers = []  

    with open(filename, 'rb') as file:
        # prime the while loop
        tempChar = file.read(1)
        
        # read file
        while tempChar:
            # convert char to 8-bit binary and append to string
            binaryContent += bin(ord(tempChar))[2:].zfill(8)

            # once binaryContent has 12 bits, convert to integer and append to list
            # and strip those 12 bits from binaryContent
            if(len(binaryContent) >= 12):
                numbers.append(int(binaryContent[0:12], 2))

                # remove the first 12 bits from binaryContent
                binaryContent = binaryContent[12:]
                
            # read 1 byte
            tempChar = file.read(1)
    return numbers

# read file in binary and convert to 12-bit ascii
def readModified(filename):
    numbers = []  

    file = open(filename, 'rb')
    rawBinary = file.read()
    # concatinate all bytes in rawBinary into one string object
    binaryString = ''.join(format(byte, '08b') for byte in rawBinary)

    for numBits in range(9,17):
        if(numBits == 9):
            offset = 1
        else:
            offset = 0

        for _ in range(0, 2**numBits-2**(numBits-1) + offset):
            if(binaryString == "" or len(binaryString) < numBits):
                break
            numbers.append(int(binaryString[:numBits], 2))
            binaryString = binaryString[numBits:]
    
    return numbers

# class to store the decimal value and number of bits of a LZW number
class LZWNode:
    def __init__(self, decimal, numBits):
        self.decimal = decimal
        self.numBits = numBits
    
    def getDecimal(self):
        return self.decimal
    
    def getNumBits(self):
        return self.numBits

In [9]:
# you will compress the file named filename, and save the compressed as filename+".lzw"
# keep the function name
def LZW_compress(fname):
    # create a dictionary
    dict_size = 256
    # make a dictionary and populate it with the ASCII values of the characters from 0 - 256
    # also note the dictionary is indexed by chars
    dictionary = {chr(i): i for i in range(dict_size)}

    # read the file
    file = open(fname, 'r')
    fileContent = file.read()

    #LZW compression algorithm
    lastWord = ""
    result = []
    for currentChar in fileContent:
        wc = lastWord + currentChar
        if wc in dictionary:
            lastWord = wc
        else:
            # check if lastWord is ASCII
            isASCII(lastWord)

            # put the code for lastWord in the result
            result.append(dictionary[lastWord])
            
            lastWord = currentChar

            # put a limit on the dictionary size to 2^12
            if(dict_size < 4096):
                # Add wc to the dictionary.
                dictionary[wc] = dict_size
                dict_size += 1
    
    # if lastWord is not empty, put it in the result list one final time         
    if lastWord:
        result.append(dictionary[lastWord])
    
    write(fname + ".lzw", result)

# keep this line 
LZW_compress(filename)

In [10]:
# you'll expand the file named filename.lzw, and save the decompressed as filename+".2"
# keep the function name
def LZW_expand(fname):
    """Decompress a list of output to a string."""
    from io import StringIO

    # check if the file has the right extension
    if not fname.endswith('.lzw'):
        raise ValueError('File must have a .lzw extension')

    # Build the dictionary.
    dict_size = 256
    # make a dictionary and populate it with the ASCII values of the characters from 0 - 256
    # also note the dictionary is indexed by integers
    dictionary = {i: chr(i) for i in range(dict_size)}

    # read the file
    LZWNumbers = read(fname) 
    
    # use StringIO, otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = StringIO()
    
    # prime the for loop with the first character
    w = chr(LZWNumbers.pop(0))
    result.write(w)

    #LZW decompression algorithm
    for number in LZWNumbers:
        # if number is in the dictionary, get the entry
        if number in dictionary:
            word = dictionary[number]

        # otherwise if number is not in the dictionary but is the very next word, special case
        elif number == dict_size: 
            word = w + w[0]
            
        # otherwise, raise an exception
        else:
            raise ValueError('Bad compressed number: %s' % number)
        
        # write the word to the result
        result.write(word)

        # Add w + the first letter of word to the dictionary.
        dictionary[dict_size] = w + word[0]
        dict_size += 1

        w = word
    
    # write the result to a file
    with open(fname[:-3] + "2", 'w') as file:
        file.write(result.getvalue())
    
# keep this line 
LZW_expand(filename + ".lzw")

In [11]:
# keep the function name
def LZW_modified_compress(fname):
    # create a dictionary
    dict_size = 256
    # make a dictionary and populate it with the ASCII values of the characters from 0 - 256
    # also note the dictionary is indexed by chars
    dictionary = {chr(i): i for i in range(dict_size)}

    # read the file
    file = open(fname, 'r')
    fileContent = file.read()

    #LZW compression algorithm
    numBits = 9
    lastWord = ""
    result = []
    for currentChar in fileContent:
        wc = lastWord + currentChar
        if wc in dictionary:
            lastWord = wc
        else:
            # check if lastWord is ASCII
            isASCII(lastWord)

            # put the code for lastWord in the result
            encodedWord = dictionary[lastWord]
            result.append(LZWNode(encodedWord, numBits))

            # Add wc to the dictionary.
            lastWord = currentChar
            
            # increment numBits if the dictionary is full
            if (dict_size >= 2**numBits and numBits < 16):
                numBits += 1
            # put a limit on the dictionary size 16 bits
            if(numBits <= 16):
                dictionary[wc] = dict_size
                dict_size += 1

    # if lastWord is not empty, put it in the result list one final time         
    if lastWord:
        encodedWord = dictionary[lastWord]
        result.append(LZWNode(encodedWord, numBits))

    # convert the list of LZWNode objects to a list of binary strings
    binaryList = []
    for element in result:
        binaryList.append(bin(element.getDecimal())[2:].zfill(element.getNumBits()))
    
    # write the binary strings to a file
    writeModified(fname + ".lzw2", binaryList)

# keep this line    
LZW_modified_compress(filename)

In [12]:
# keep the function name
def LZW_modified_expand(fname):
    """Decompress a list of output ks to a string."""
    from io import StringIO

    if not fname.endswith('.lzw2'):
        raise ValueError('File must have a .lzw2 extension')

    # Build the dictionary.
    dict_size = 256
    # make a dictionary and populate it with the ASCII values of the characters from 0 - 256
    # also note the dictionary is indexed by integers
    dictionary = {i: chr(i) for i in range(dict_size)}

    # read the file
    LZWNumbers = readModified(fname)

    # use StringIO, otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = StringIO()
    
    # prime the for loop with the first character
    w = chr(LZWNumbers.pop(0))
    result.write(w)
    
    #LZW decompression algorithm
    for number in LZWNumbers:
        # if number is in the dictionary, get the entry
        if number in dictionary:
            word = dictionary[number]

        # otherwise if nnumber is not in the word but is the very next word, special case
        elif number == dict_size: 
            word = w + w[0]
            
        # otherwise if the number isn't the next word to guess, raise an exception
        else:
            raise ValueError('Bad compressed number: %s' % number)
        
        # write the word to the result
        result.write(word)
        
        # Add w + the first letter of word to the dictionary.
        dictionary[dict_size] = w + word[0]
        dict_size += 1

        w = word

    
    # write the result to a file
    with open(fname[:-4] + "2M", 'w') as file:
        file.write(result.getvalue())
    
# keep this line    
LZW_modified_expand(filename+".lzw2")